In [1]:
import numpy as np, pandas as pd
from tqdm import tqdm_notebook

In [2]:
def extr_data_txt(path):
    with open(path, "r") as f:
        data = []
        for itr, line in tqdm_notebook(enumerate(f)):
            data.append(line.split())
    return data[2:]

In [3]:
train_test_valid_anot = pd.DataFrame(extr_data_txt('data/Eval/list_eval_partition.txt'), 
                                     columns=['Path', 'type'])
train_test_valid_anot.to_csv('data/Anno/train_test_valid_anot.csv' ,index=False)

In [4]:
categories_img = pd.DataFrame(extr_data_txt('data/Anno/list_category_img.txt'), 
                          columns=['Path', 'cat'])
categories_img.to_csv('data/Anno/categories_img.csv' ,index=False)

In [5]:
box_img = pd.DataFrame(extr_data_txt('data/Anno/list_bbox.txt'), 
                          columns=['Path', 'x1', 'y1', 'x2', 'y2'])
box_img.to_csv('data/Anno/bbox_img.csv' ,index=False)

In [ ]:
with open('data/Anno/list_attr_img.txt', "r") as f:
    data = []
    for itr, line in tqdm_notebook(enumerate(f)):
            if itr >= 2:
                line = line.split()
                tmp = [np.int8(line[i]) for i in range(1, 1001)]
                tmp.append(line[0])
                data.append(tmp)

In [ ]:
np.savez('data/Anno/Atr_img.txt', data)